In [6]:
import os
from dotenv import load_dotenv
from langfuse.callback import CallbackHandler
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langfuse.callback import CallbackHandler
from langchain_openai import ChatOpenAI
import pandas as pd
import sys
sys.path.append('../')
import prompts as p




# Load the .env file
load_dotenv()

# Access the environment variables
os.environ["LANGFUSE_PUBLIC_KEY"] = os.environ.get("LANGFUSE_PUBLIC_KEY")
os.environ["LANGFUSE_SECRET_KEY"] = os.environ.get("LANGFUSE_SECRET_KEY")
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

handler = CallbackHandler()
handler.auth_check()

DATA_PATH = "../../../Datasets/MAMS-ATSA/Downsampled/test/test_single_row.csv"
OUTPUT_PATH = "../../../Datasets/Evaluations/Sentiment Analysis/Sentiment_Analysis_Zero_shot_gpt-4-turbo-preview.csv"

MODEL = "gpt-4"
MAX_TOKENS = 15
TEMP = 0



In [2]:
df = pd.read_csv(DATA_PATH)
prompt_name = "prompt_single_term_zeroshot"
prompt = p.prompt_single_term_zeroshot


df['polarity_pred'] = None
df['prompt_name'] = None
df['prompt'] = None
print(df.head())


   text_id  term_id                                               text  \
0        0        0  Amusing details distinguish desserts, from dul...   
1        0        1  Amusing details distinguish desserts, from dul...   
2        0        2  Amusing details distinguish desserts, from dul...   
3        0        3  Amusing details distinguish desserts, from dul...   
4        1        4  The server was so busy the night we visited th...   

                          term  polarity polarity_pred prompt_name prompt  
0                     desserts  positive          None        None   None  
1     dulce de leche ice-cream   neutral          None        None   None  
2  chocolate sauce tic-tac-toe   neutral          None        None   None  
3            poached pineapple   neutral          None        None   None  
4                       server  negative          None        None   None  


In [4]:
def execute(df, prompt, prompt_name, start = 0):
    if(MODEL == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name = MODEL, temperature = TEMP, max_tokens=MAX_TOKENS, timeout=10)
    else:
        llm = ChatOpenAI(model_name = MODEL, temperature = TEMP, max_tokens=MAX_TOKENS, timeout=10)
    for i in range(start, len(df)):
        if(df["polarity_pred"][i] != None and df["polarity_pred"][i] != "error"):
            continue
        #time.sleep(0.1)
        print(i)
        chain = LLMChain(llm=llm, prompt=prompt, callbacks=[handler])
        input_text = df["text"][i]
        entity = df["term"][i]
        try: 
            
            result = chain.run(input_text = input_text, entity = entity, callbacks=[handler])
        except Exception as e:
            print(e)
            df.loc[i, 'polarity_pred'] =  'error'
            df.loc[i, 'prompt_name'] = prompt_name
            df.loc[i, 'prompt'] = chain.prompt.format_prompt(input_text = input_text, entity = entity).text
            continue
        handler.langfuse.flush()
        prompt_text = chain.prompt.format_prompt(input_text = input_text, entity = entity).text
        if (i < 5):
            print(prompt_text)
            print("\n")
            print(result)
            print("\n")

        if (i % 50 == 0):
            print(str(i) + " of  " + str(len(df)))

        if ("positive" in result.lower()):
            y_pred = 'positive'
        elif ("negative" in result.lower()):
            y_pred = 'negative'
        elif ("neutral" in result.lower()):
            y_pred = 'neutral'
        else:
            y_pred = 'error'

        df.loc[i, 'polarity_pred'] =  y_pred
        df.loc[i, 'prompt_name'] = prompt_name
        df.loc[i, 'prompt'] = prompt_text
    return df


df = execute(df, prompt, prompt_name)
    



1
Amusing details distinguish desserts, from dulce de leche ice-cream profiteroles dotting a chocolate sauce tic-tac-toe board, to coconut custard surrounded by a sea of Malibu-rum gelee and poached pineapple.

    What is the sentiment on 'dulce de leche ice-cream'? Only respond with "positive", "negative" or "neutral" as one word.


positive


2
Amusing details distinguish desserts, from dulce de leche ice-cream profiteroles dotting a chocolate sauce tic-tac-toe board, to coconut custard surrounded by a sea of Malibu-rum gelee and poached pineapple.

    What is the sentiment on 'chocolate sauce tic-tac-toe'? Only respond with "positive", "negative" or "neutral" as one word.


positive


3
Amusing details distinguish desserts, from dulce de leche ice-cream profiteroles dotting a chocolate sauce tic-tac-toe board, to coconut custard surrounded by a sea of Malibu-rum gelee and poached pineapple.

    What is the sentiment on 'poached pineapple'? Only respond with "positive", "negative"

In [5]:
df

,text_id,term_id,text,term,polarity,polarity_pred,prompt_name,prompt
0,0,0,"Amusing details distinguish desserts, from dul...",desserts,positive,positive,prompt_3_zeroshot_single_term,"Amusing details distinguish desserts, from dul..."
1,0,1,"Amusing details distinguish desserts, from dul...",dulce de leche ice-cream,neutral,positive,prompt_3_zeroshot_single_term,"Amusing details distinguish desserts, from dul..."
2,0,2,"Amusing details distinguish desserts, from dul...",chocolate sauce tic-tac-toe,neutral,positive,prompt_3_zeroshot_single_term,"Amusing details distinguish desserts, from dul..."
3,0,3,"Amusing details distinguish desserts, from dul...",poached pineapple,neutral,positive,prompt_3_zeroshot_single_term,"Amusing details distinguish desserts, from dul..."
4,1,4,The server was so busy the night we visited th...,server,negative,negative,prompt_3_zeroshot_single_term,The server was so busy the night we visited th...
...,...,...,...,...,...,...,...,...
446,169,446,The server came by only once to pour additiona...,server,negative,negative,prompt_3_zeroshot_single_term,The server came by only once to pour additiona...
447,169,447,The server came by only once to pour additiona...,wine,neutral,neutral,prompt_3_zeroshot_single_term,The server came by only once to pour additiona...
448,169,448,The server came by only once to pour additiona...,the table,neutral,negative,prompt_3_zeroshot_single_term,The server came by only once to pour additiona...
449,169,449,The server came by only once to pour additiona...,fish,neutral,negative,prompt_3_zeroshot_single_term,The server came by only once to pour additiona...


In [7]:
df.to_csv(OUTPUT_PATH, index = True)